In [ ]:
import polars as pl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import aquarel as aq
from ineqpy.inequality import gini
import scipy.stats as st
import pyalex as alex
alex.config.email = "noah0roussel01980@gmail.com"
works = pl.read_csv("../data/2_extracted_works/works_q1.csv")
works = works.filter(pl.col("year") != 2025)

year_begin = 1920
year_end = 2024
works = works.with_columns(
    age=2025 - pl.col("year")
)


In [ ]:
works_bjp = pl.read_csv("../data/2_extracted_works/works_bjp.csv")
works_bjp = works_bjp.filter(pl.col("year") != 2025)

In [ ]:
cbc_per_year = (
    works
    .select(
        [pl.col("year"), pl.col("cited_by_count"), pl.col("title")]
    )
    .group_by(
        pl.col("year"),
        maintain_order=True
    )
    .agg(
        pl.col("cited_by_count").mean().name.prefix("mean_")
    )
) 

works = works.join(
    cbc_per_year,
    on="year",
    how="left"
) 

works = (
    works
    .with_columns(
        mncs = pl.col("cited_by_count") / pl.col("mean_cited_by_count")
    )
    .drop("mean_cited_by_count")
)
cols_authors = [col for col in works.columns if col.startswith("author_")]

works = works.with_columns(
    authors_count = sum(
        [pl.col(col).is_not_null().cast(pl.Int8) for col in cols_authors]
    )

)

works = works.with_columns(
    title = pl.col("title").fill_null("").str.to_lowercase(),
    abstract = pl.col("abstract").fill_null("").str.to_lowercase()
)

works = works.with_columns( #remove abstract, add words : survey, overview, state of?
    review = (
        pl.col("title").str.contains("review") |
        pl.col("abstract").str.contains("review")
    ),
    meta_analysis = (
        pl.col("title").str.contains("meta[\u00AD-]?analysis") |
        pl.col("abstract").str.contains("meta[\u00AD-]?analysis")
    )
)

works = works.with_columns([
    pl.when(pl.col("countries_distinct_count").is_null() | pl.col("countries_distinct_count").is_nan() | (pl.col("countries_distinct_count") == 0))
      .then(1)
      .otherwise(pl.col("countries_distinct_count"))
      .alias("countries_distinct_count")
])
works = works.with_columns([
    pl.when(pl.col("institutions_distinct_count").is_null() | pl.col("institutions_distinct_count").is_nan() | (pl.col("institutions_distinct_count") == 0))
      .then(1)
      .otherwise(pl.col("institutions_distinct_count"))
      .alias("institutions_distinct_count")
])

works

In [ ]:
year_begin = 1920
year_end = 2024
works_bjp = works_bjp.with_columns(
    age=2025 - pl.col("year")
)

cbc_per_year = (
    works
    .group_by("year")
    .agg(
        pl.col("cited_by_count").mean().alias("mean_cited_by_count")
    )
)


works_bjp = works_bjp.join(
    cbc_per_year,
    on="year",
    how="left"
)


works_bjp = (
    works_bjp
    .with_columns(
        mncs = pl.col("cited_by_count") / pl.col("mean_cited_by_count")
    )
    .drop("mean_cited_by_count")
)

bjp_cols_authors = [col for col in works_bjp.columns if col.startswith("author_")]

works_bjp = works_bjp.with_columns(
    authors_count = sum(
        [pl.col(col).is_not_null().cast(pl.Int8) for col in bjp_cols_authors]
    )

)

works_bjp = works_bjp.with_columns(
    title = pl.col("title").fill_null("").str.to_lowercase(),
    abstract = pl.col("abstract").fill_null("").str.to_lowercase()
)

works_bjp = works_bjp.with_columns( #remove abstract, add words : survey, overview, state of?
    review = (
        pl.col("title").str.contains("review") |
        pl.col("abstract").str.contains("review")
    ),
    meta_analysis = (
        pl.col("title").str.contains("meta[\u00AD-]?analysis") |
        pl.col("abstract").str.contains("meta[\u00AD-]?analysis")
    )
)

works_bjp = works_bjp.with_columns([
    pl.when(pl.col("countries_distinct_count").is_null() | pl.col("countries_distinct_count").is_nan() | (pl.col("countries_distinct_count") == 0))
      .then(1)
      .otherwise(pl.col("countries_distinct_count"))
      .alias("countries_distinct_count")
])
works_bjp = works_bjp.with_columns([
    pl.when(pl.col("institutions_distinct_count").is_null() | pl.col("institutions_distinct_count").is_nan() | (pl.col("institutions_distinct_count") == 0))
      .then(1)
      .otherwise(pl.col("institutions_distinct_count"))
      .alias("institutions_distinct_count")
])


bjp_group_references_mean = (
    works_bjp.lazy()  
    .filter(~pl.col("review") & ~pl.col("meta_analysis"))
    .group_by("referenced_works_count")
    .agg([
        pl.col("mncs").mean().alias("mncs"),
        pl.len().alias("count")
    ])
    .sort("referenced_works_count")
    .collect()  
)

works_bjp

In [ ]:


country_cols = [c for c in works.columns if c.startswith("country_")]


top10 = (
    works
    .select(["title", "cited_by_count"] + country_cols)
    .sort("cited_by_count", descending=True)
    .head(10)
)

top10_list = (
    top10
    .with_columns(
        pl.when(
            pl.any_horizontal(
                [pl.col(c).is_not_null() for c in country_cols]
            )
        )
        .then(pl.concat_list(country_cols))
        .otherwise(pl.lit(["Unknown"]))
        .alias("countries")
    )
    .select(["title", "cited_by_count", "countries"])
)


exploded = top10_list.explode("countries").drop_nulls()

country_prop = (
    exploded
    .group_by("title", "countries")
    .agg(pl.count().alias("count"))
    .group_by("title")
    .agg(
        [
            pl.col("countries"),
            (pl.col("count") / pl.col("count").sum()).alias("proportion")
        ]
    )
)


with_citations = country_prop.join(
    top10_list.select(["title", "cited_by_count"]),
    on="title",
    how="left"
).with_columns(
    (pl.col("proportion") * pl.col("cited_by_count")).alias("cit_weight")
)


pdf = with_citations.explode(["countries", "cit_weight"]).to_pandas()
df_stack = pdf.pivot_table(
    index="title",
    columns="countries",
    values="cit_weight",
    fill_value=0
)

print(df_stack)


def shorten_title(title):
    parts = title.split()
    if len(parts) <= 2:
        return title
    return f"{parts[0]} [...] {parts[-1]}"

pdf = with_citations.explode(["countries", "cit_weight"]).to_pandas()
df_stack = pdf.pivot_table(
    index="title",
    columns="countries",
    values="cit_weight",
    fill_value=0
)

df_stack = df_stack.reindex(top10["title"].to_list())


df_stack.index = [shorten_title(t) for t in df_stack.index]

top10_list.to_pandas()

print(top10_list["title"].to_list()
,top10_list["cited_by_count"].to_list())




In [ ]:
country_cols = [c for c in works_bjp.columns if c.startswith("country_")]

top10 = (
    works_bjp
    .select(["title", "cited_by_count"] + country_cols)
    .sort("cited_by_count", descending=True)
    .head(10)
)

top10_list = (
    top10
    .with_columns(
        pl.concat_list(country_cols).alias("countries")
    )
    .select(["title", "cited_by_count", "countries"])
)

exploded = top10_list.explode("countries").drop_nulls()

country_prop = (
    exploded
    .group_by("title", "countries")
    .agg(pl.count().alias("count"))
    .group_by("title")
    .agg(
        [
            pl.col("countries"),
            (pl.col("count") / pl.col("count").sum()).alias("proportion")
        ]
    )
)

with_citations = country_prop.join(
    top10_list.select(["title", "cited_by_count"]),
    on="title",
    how="left"
).with_columns(
    (pl.col("proportion") * pl.col("cited_by_count")).alias("cit_weight")
)

pdf = with_citations.explode(["countries", "cit_weight"]).to_pandas()
df_stack_bjp = pdf.pivot_table(
    index="title",
    columns="countries",
    values="cit_weight",
    fill_value=0
)
print(df_stack_bjp)

def shorten_title(title):
    parts = title.split()
    if len(parts) <= 2:
        return title
    if parts[-1] == "vivo</i>":
        return f"{parts[0]}[...] vivo"
    if parts [-1] == "δ<sup>9</sup>‐tetrahydrocannabivarin":
        return f"{parts[0]}[...] tetrahydrocannabivarin"
    return f"{parts[0]} [...] {parts[-1]}"

df_stack_bjp.index = [shorten_title(t) for t in df_stack_bjp.index]

title_order = [shorten_title(t) for t in top10["title"].to_list()]
df_stack_bjp = df_stack_bjp.loc[title_order]

top10_list.to_pandas()

print(top10_list["title"].to_list(),
top10_list["cited_by_count"].to_list())

In [ ]:
all_categories = sorted(
    set(df_stack.columns) | set(df_stack_bjp.columns)
)

base_colors = plt.rcParams["axes.prop_cycle"].by_key()["color"]

palette = {}
i = 0
for c in all_categories:
    if c == "Unknown":
        palette[c] = "lightgrey"
    else:
        palette[c] = base_colors[i % len(base_colors)]
        i += 1


plt.figure(figsize=(12,7))
colors_1 = [palette[c] for c in df_stack.columns]

df_stack.plot(
    kind="bar",
    stacked=True,
    figsize=(12,7),
    color=colors_1
)

plt.ylabel("Number of Citations", fontsize=14)
plt.xlabel("Work", fontsize=14)
plt.title("Top 10 Q1 Cited Works – Country Composition of Authors", fontsize=16)
plt.legend(title="Country", fontsize=10, bbox_to_anchor=(1.05,1), loc="upper left")
plt.grid(alpha=0.5)
plt.tight_layout()

ax = plt.gca()
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
plt.show()


In [ ]:


plt.figure(figsize=(12,7))
colors_2 = [palette[c] for c in df_stack_bjp.columns]

df_stack_bjp.plot(
    kind="bar",
    stacked=True,
    figsize=(12,7),
    color=colors_2
)

plt.ylabel("Number of Citations", fontsize=14)
plt.xlabel("Work", fontsize=14)
plt.title("Top 10 BJP Cited Works – Country Composition of Authors", fontsize=16)
plt.legend(title="Country", fontsize=10, bbox_to_anchor=(1.05,1), loc="upper left")
plt.grid(alpha=0.5)
plt.tight_layout()

ax = plt.gca()
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
plt.show()

In [ ]:
group_works = (
    works
    .group_by("year")
    .agg([
        (pl.col("review").mean() * 100).alias("review_pct"),
        (pl.col("meta_analysis").mean() * 100).alias("meta_pct")
    ])
    .sort("year")
)

group_works_bjp = (
    works_bjp
    .group_by("year")
    .agg([
        (pl.col("review").mean() * 100).alias("review_pct"),
        (pl.col("meta_analysis").mean() * 100).alias("meta_pct")
    ])
    .sort("year")
)



In [ ]:
years = group_works["year"].to_numpy()
review_pct = group_works["review_pct"].to_numpy()
meta_pct  = group_works["meta_pct"].to_numpy()

years_bjp = group_works_bjp["year"].to_numpy()
review_pct_bjp = group_works_bjp["review_pct"].to_numpy()
meta_pct_bjp  = group_works_bjp["meta_pct"].to_numpy()

with aq.load_theme("scientific"):
    plt.figure(figsize=(10,5))

    plt.plot(years, review_pct, label="Q1 Review %")

    plt.plot(years_bjp, review_pct_bjp, label="BJP Review %")

    plt.xlabel("Year")
    plt.ylabel("Percentage (%)")
    plt.title("Percentage of Review Papers per Year - BJP vs Q1 Journals")
    plt.grid(True)
    plt.xlim(1950, 2024)

    plt.legend()

    plt.show()



In [ ]:


with aq.load_theme("scientific"):
    plt.figure(figsize=(10,5))

    plt.plot(years, meta_pct, label="Q1 Meta-analysis %")

    plt.plot(years_bjp, meta_pct_bjp, label="BJP Meta-analysis %")

    plt.xlabel("Year")
    plt.ylabel("Percentage (%)")
    plt.title("Percentage of Meta-analysis Papers per Year - BJP vs Q1 Journals")
    plt.grid(True)
    plt.xlim(1950, 2024)

    plt.legend()

    plt.show()

In [ ]:
cols_to_stats = [
    "cited_by_count",
    "countries_distinct_count",
    "institutions_distinct_count",
    "referenced_works_count",
    "authors_count",
    "mncs"
]

agg_exprs = []

for col in cols_to_stats:
    agg_exprs.append(pl.mean(col).alias(f"{col}_mean"))
    agg_exprs.append(pl.median(col).alias(f"{col}_median"))
    agg_exprs.append(pl.col(col).quantile(0.025).alias(f"{col}_p2_5"))
    agg_exprs.append(pl.col(col).quantile(0.975).alias(f"{col}_p97_5"))
    agg_exprs.append(pl.sum(col).alias(f"{col}_sum"))


In [ ]:

cited_cols = [f"cited_by_count_{year}" for year in range(2012, 2025) if f"cited_by_count_{year}" in works.columns]

for col_name in cited_cols:
    agg_exprs.append(pl.sum(col_name).alias(f"{col_name}_sum"))

if cited_cols:
    agg_exprs.append(
        sum([pl.col(col_name) for col_name in cited_cols]).sum().alias("cited_by_count_since_2012")
    )
    
agg_exprs.append(pl.count().alias("count"))
agg_exprs.append(pl.col("year").min().alias("first_year"))

In [ ]:
group_year_bjp = (
    works_bjp.group_by(
        by=pl.col("year"),
        maintain_order=True
    )
    .agg(agg_exprs)
)

group_year_q1 = (
    works.group_by(
        by=pl.col("year"),
        maintain_order=True
    )
    .agg(agg_exprs)
)

group_journals = (
    works.group_by(
        by=pl.col("journal"),
        maintain_order=True
    )
    .agg(agg_exprs)
)

group_year_q1 = group_year_q1.sort("by", descending=False)
group_year_bjp = group_year_bjp.sort("by", descending=False)
print(group_year_q1)


In [ ]:
group_journals.filter(pl.col("by") == "British Journal of Pharmacology")


In [ ]:
journals_recent = group_journals.sort("first_year", descending=False)

journals_recent

In [ ]:
import math

df = journals_recent.sort("first_year")

names = df["by"].to_list()
years = df["first_year"].to_list()

total = len(df)
blocks = 3
block_size = math.ceil(total / blocks)

def create_table(ax, sub_names, sub_years, title):
    data = list(zip(sub_names, sub_years))
    ax.axis("off")
    table = ax.table(
        cellText=data,
        colLabels=["Journal", "First Year"],
        loc="center",
        cellLoc="left"
    )
    table.auto_set_font_size(False)
    table.set_fontsize(8)
    table.scale(1, 1.3)
    ax.set_title(title, fontsize=12, pad=10)

for part in range(blocks):
    start = part * block_size
    end = min((part + 1) * block_size, total)
    
    fig, ax = plt.subplots(figsize=(12, len(range(start, end))*0.35 + 2))
    create_table(ax, names[start:end], years[start:end], f"Journals First Publication — Part {part+1}")
    plt.tight_layout()
    plt.show()


In [ ]:
years = range(group_journals["first_year"].min(), 2025)
active_counts = [
    (group_journals["first_year"] <= y).sum()
    for y in years
]

plt.figure(figsize=(10,5))
plt.plot(years, active_counts)
plt.xlabel("Year")
plt.ylabel("Number of Active Journals")
plt.title("Evolution of Active Q1 Journals over time")
plt.grid(True)
plt.xlim(1950, 2024)
plt.show()


In [ ]:
journals_top20_publications = group_journals.sort("count", descending=True).head(20)
journals_top20cited = group_journals.sort("cited_by_count_sum", descending=True).head(20)
journals_top20cited_mean = group_journals.sort("cited_by_count_mean", descending=True).head(20)
journals_top20cited_since2012 = group_journals.sort("cited_by_count_since_2012", descending=True).head(20)
journals_top20_mncs = group_journals.sort("mncs_mean", descending=True).head(20)

journals_top20_countrydiverse = group_journals.sort("countries_distinct_count_mean", descending=True).head(20)
journals_top20_institutionsdiverse = group_journals.sort("institutions_distinct_count_mean", descending=True).head(20)
journals_top20_authorsdiverse = group_journals.sort("authors_count_mean", descending=True).head(20)
journals_top20_references = group_journals.sort("referenced_works_count_mean", descending=True).head(20)

journals_top20_countrydiverse_sum = group_journals.sort("countries_distinct_count_sum", descending=True).head(20)
journals_top20_institutionsdiverse_sum = group_journals.sort("institutions_distinct_count_sum", descending=True).head(20)
journals_top20_authorsdiverse_sum = group_journals.sort("authors_count_sum", descending=True).head(20)
journals_top20_references_sum = group_journals.sort("referenced_works_count_sum", descending=True).head(20)



In [ ]:
def rank_of_bjp(df, column, ascending=False):
    sorted_df = df.sort(column, descending=not ascending)
    ranked = sorted_df.with_row_index("rank", offset=1)
    return ranked.filter(pl.col("by") == "British Journal of Pharmacology")[["rank", column]]

print("Rank by publications:", rank_of_bjp(group_journals, "count"))
print("Rank by total citations:", rank_of_bjp(group_journals, "cited_by_count_sum"))
print("Rank by mean citations:", rank_of_bjp(group_journals, "cited_by_count_mean"))
print("Rank by citations since 2012:", rank_of_bjp(group_journals, "cited_by_count_since_2012"))
print("Rank by country diversity:", rank_of_bjp(group_journals, "countries_distinct_count_mean"))
print("Rank by institution diversity:", rank_of_bjp(group_journals, "institutions_distinct_count_mean"))
print("Rank by author diversity:", rank_of_bjp(group_journals, "authors_count_mean"))
print("Rank by references count:", rank_of_bjp(group_journals, "referenced_works_count_mean"))
print("Rank by MNCS:", rank_of_bjp(group_journals, "mncs_mean"))
print("Rank by total referencess:", rank_of_bjp(group_journals, "referenced_works_count_sum"))
print("Rank by total countries:", rank_of_bjp(group_journals, "countries_distinct_count_sum"))
print("Rank by total authors:", rank_of_bjp(group_journals, "authors_count_sum"))
print("Rank by total institutions:", rank_of_bjp(group_journals, "institutions_distinct_count_sum"))



In [ ]:

with aq.load_theme("scientific"):
    plt.figure(figsize=(10,15))
    sns.barplot(x="count", y="by", data=journals_top20_publications, palette="viridis")
    plt.xlabel("Number of Publications")
    plt.ylabel("Journal")
    plt.title("Publications by Journal - Top 20 Q1 Journals")
    plt.tight_layout()
    plt.show()


In [ ]:

with aq.load_theme("scientific"):
    plt.figure(figsize=(10,15))
    sns.barplot(x="cited_by_count_sum", y="by", data=journals_top20cited, palette="viridis")
    plt.xlabel("Number of Citations")
    plt.ylabel("Journal")
    plt.title("Citations by Journal - Top 20 Q1 Journals")
    plt.tight_layout()
    plt.show()


In [ ]:

with aq.load_theme("scientific"):
    plt.figure(figsize=(10,15))
    sns.barplot(x="cited_by_count_mean", y="by", data=journals_top20cited_mean, palette="viridis")
    plt.xlabel("Number of Citations")
    plt.ylabel("Journal")
    plt.title("Mean Citations by Work - Top 20 Q1 Journals")
    plt.tight_layout()
    plt.show()


In [ ]:

with aq.load_theme("scientific"):
    plt.figure(figsize=(10,15))
    sns.barplot(x="cited_by_count_since_2012", y="by", data=journals_top20cited_since2012, palette="viridis")
    plt.xlabel("Number of Citations")
    plt.ylabel("Journal")
    plt.title("Citations since 2012 by Journal - Top 20 Q1 Journals")
    plt.tight_layout()
    plt.show()


In [ ]:

with aq.load_theme("scientific"):
    plt.figure(figsize=(10,15))
    sns.barplot(x="referenced_works_count_mean", y="by", data=journals_top20_references, palette="viridis")
    plt.xlabel("Number of References")
    plt.ylabel("Journal")
    plt.title("Mean References by Journal - Top 20 Q1 Journals")
    plt.tight_layout()
    plt.show()


In [ ]:

with aq.load_theme("scientific"):
    plt.figure(figsize=(10,15))
    sns.barplot(x="referenced_works_count_sum", y="by", data=journals_top20_references_sum, palette="viridis")
    plt.xlabel("Number of References")
    plt.ylabel("Journal")
    plt.title("Total References by Journal - Top 20 Q1 Journals")
    plt.tight_layout()
    plt.show()


In [ ]:

with aq.load_theme("scientific"):
    plt.figure(figsize=(10,15))
    sns.barplot(x="authors_count_sum", y="by", data=journals_top20_authorsdiverse_sum, palette="viridis")
    plt.xlabel("Number of Authors")
    plt.ylabel("Journal")
    plt.title("Total Distinct Authors by Journal - Top 20 Q1 Journals")
    plt.tight_layout()
    plt.show()


In [ ]:

with aq.load_theme("scientific"):
    plt.figure(figsize=(10,15))
    sns.barplot(x= "countries_distinct_count_sum", y="by", data=journals_top20_countrydiverse_sum, palette="viridis")
    plt.xlabel("Number of Countries")
    plt.ylabel("Journal")
    plt.title("Total Distinct Countries by Journal - Top 20 Q1 Journals")
    plt.tight_layout()
    plt.show()


In [ ]:

with aq.load_theme("scientific"):
    plt.figure(figsize=(10,15))
    sns.barplot(x="institutions_distinct_count_sum", y="by", data=journals_top20_institutionsdiverse_sum, palette="viridis")
    plt.xlabel("Number of Institutions")
    plt.ylabel("Journal")
    plt.title("Total Distinct Institutions by Journal - Top 20 Q1 Journals")
    plt.tight_layout()
    plt.show()


In [ ]:

with aq.load_theme("scientific"):
    plt.figure(figsize=(10,15))
    sns.barplot(x="mncs_mean", y="by", data=journals_top20_mncs, palette="viridis")
    plt.xlabel("MNCS")
    plt.ylabel("Journal")
    plt.title("Mean MNCS by Journal - Top 20 Q1 Journals")
    plt.tight_layout()
    plt.show()


In [ ]:

with aq.load_theme("scientific"):
    plt.figure(figsize=(10,15))
    sns.barplot(x="authors_count_mean", y="by", data=journals_top20_authorsdiverse, palette="viridis")
    plt.xlabel("Number of Authors")
    plt.ylabel("Journal")
    plt.title("Mean Authors by Journal - Top 20 Q1 Journals")
    plt.tight_layout()
    plt.show()


In [ ]:

with aq.load_theme("scientific"):
    plt.figure(figsize=(10,15))
    sns.barplot(x="countries_distinct_count_mean", y="by", data=journals_top20_countrydiverse, palette="viridis")
    plt.xlabel("Number of Countries")
    plt.ylabel("Journal")
    plt.title("Mean Distinct Countries by Journal - Top 20 Q1 Journals")
    plt.tight_layout()
    plt.show()


In [ ]:

with aq.load_theme("scientific"):
    plt.figure(figsize=(10,15))
    sns.barplot(x="institutions_distinct_count_mean", y="by", data=journals_top20_institutionsdiverse, palette="viridis")
    plt.xlabel("Number of Institutions")
    plt.ylabel("Journal")
    plt.title("Mean Distinct institutions by Journal - Top 20 Q1 Journals")
    plt.tight_layout()
    plt.show()


In [ ]:
def gini(array):
    array = np.sort(array)
    n = len(array)
    if n == 0:
        return np.nan
    cumx = np.cumsum(array)
    return (n + 1 - 2 * np.sum(cumx) / cumx[-1]) / n if cumx[-1] > 0 else 0

def bootstrap_gini(data, n_boot=500):
    ginis = []
    n = len(data)
    if n == 0:
        return np.nan, np.nan, np.nan
    for _ in range(n_boot):
        sample = np.random.choice(data, size=n, replace=True)
        ginis.append(gini(sample))
    return np.mean(ginis), np.percentile(ginis, 2.5), np.percentile(ginis, 97.5)

In [ ]:
ginis_q1, low_ginis_q1, high_ginis_q1 = [], [], []
for year in range(year_begin, year_end + 1):
    works_year = works.filter(pl.col("year") == year)
    cited = np.array(works_year["cited_by_count"].to_list())
    mean_g, low_g, high_g = bootstrap_gini(cited, n_boot=500)
    ginis_q1.append(mean_g)
    low_ginis_q1.append(low_g)
    high_ginis_q1.append(high_g)

ginis_bjp, low_ginis_bjp, high_ginis_bjp = [], [], []
for year in range(year_begin, year_end + 1):
    works_year = works_bjp.filter(pl.col("year") == year)
    cited = np.array(works_year["cited_by_count"].to_list())
    mean_g, low_g, high_g = bootstrap_gini(cited, n_boot=500)
    ginis_bjp.append(mean_g)
    low_ginis_bjp.append(low_g)
    high_ginis_bjp.append(high_g)
    
print(ginis_q1)

In [ ]:
x = list(range(year_begin, year_end + 1))

with aq.load_theme("scientific"):
    plt.figure(figsize=(7.8, 6.6))

    plt.plot(x, ginis_q1, color="#3944f3", label="Q1 works")
    plt.fill_between(x, low_ginis_q1, high_ginis_q1, color="#3944f3", alpha=0.2)

    plt.plot(x, ginis_bjp, color="#f39c12", label="BJP works")
    plt.fill_between(x, low_ginis_bjp, high_ginis_bjp, color="#f39c12", alpha=0.2)

    plt.xlabel("Year of publication", fontsize=16)
    plt.ylabel("Gini coefficient of citation share", fontsize=16)
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    plt.legend(fontsize=14)
    plt.xlim(1950, 2024)
    plt.title("Gini Coefficient of Citation Share Over Time - Q1 vs BJP", fontsize=18)
    plt.show()

works

In [ ]:
q1_mncs = works["mncs"].to_numpy()
bjp_mncs = works_bjp["mncs"].to_numpy()

bins = np.linspace(0, 3, 31)  
q1_counts, edges = np.histogram(q1_mncs, bins=bins)
bjp_counts, _ = np.histogram(bjp_mncs, bins=bins)

q1_counts_norm = q1_counts / q1_counts.sum()
bjp_counts_norm = bjp_counts / bjp_counts.sum()

with aq.load_theme("scientific"):
    plt.figure(figsize=(8, 6))
    plt.bar(
        edges[:-1],
        q1_counts_norm,
        width=np.diff(edges),
        alpha=0.5,
        color="tab:blue",
        label="Q1"
    )
    plt.bar(
        edges[:-1],
        bjp_counts_norm,
        width=np.diff(edges),
        alpha=0.5,
        color="tab:orange",
        label="BJP"
    )
    plt.xlabel("MNCS value", fontsize=16)
    plt.ylabel("Frequency of works", fontsize=16)
    plt.xlim(0, 3)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    plt.legend(fontsize=14)
    plt.title("MNCS Distribution - Q1 vs BJP", fontsize=18)
    plt.show()


In [ ]:
group_year_count = (
    works
    .group_by("year", maintain_order=True)
    .agg(pl.len().alias("count"))
    .with_columns((pl.col("count") / pl.col("count").max()).alias("normalized_publications"))
)
bjp_group_year_count = (
    works_bjp
    .group_by("year", maintain_order=True)
    .agg(pl.len().alias("count"))
    .with_columns((pl.col("count") / pl.col("count").max()).alias("normalized_publications"))
)

group_year_count = group_year_count.sort("year",descending=False)
bjp_group_year_count = bjp_group_year_count.sort("year",descending=False)

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(
    group_year_count["year"].to_numpy(),
    group_year_count["normalized_publications"].to_numpy(),
    label="Q1 Journal",
    color="tab:blue",
    linewidth=2
)
plt.plot(
    bjp_group_year_count["year"].to_numpy(),
    bjp_group_year_count["normalized_publications"].to_numpy(),
    label="BJP Journal",
    color="tab:orange",
    linewidth=2
)
plt.xlabel("Year of publication", fontsize=14)
plt.ylabel("Normalized number of publications (0-1)", fontsize=14)
plt.title("Normalized Publication Trends Over Time - Q1 vs BJP", fontsize=16)
plt.legend(fontsize=12)
plt.grid(alpha=0.3)
plt.xlim(1950, 2024)
plt.ylim(0, 1.05)
plt.show()

In [ ]:
group_year_sum = (
    works
    .group_by("year", maintain_order=True)
    .agg(
        pl.all().exclude("year").sum()
    )
)


bjp_group_year_sum = (
    works_bjp
    .group_by("year", maintain_order=True)
    .agg(
        pl.all().exclude("year").sum()
    )
)

group_year_sum = group_year_sum.sort("year",descending=False)
bjp_group_year_sum = bjp_group_year_sum.sort("year",descending=False)

In [ ]:
group_year_sum = group_year_sum.with_columns(
    (pl.col("cited_by_count") / pl.col("cited_by_count").max()).alias("normalized"),
    (pl.col("referenced_works_count") / pl.col("referenced_works_count").max()).alias("normalized_references")
)

bjp_group_year_sum = bjp_group_year_sum.with_columns(
    (pl.col("cited_by_count") / pl.col("cited_by_count").max()).alias("normalized"),
    (pl.col("referenced_works_count") / pl.col("referenced_works_count").max()).alias("normalized_references")
)

plt.figure(figsize=(10, 6))


plt.plot(
    group_year_sum["year"].to_numpy(),
    group_year_sum["cited_by_count"].to_numpy(), #or normalized
    label="Q1",
    color="tab:blue",
    linewidth=2
)

plt.plot(
    bjp_group_year_sum["year"].to_numpy(),
    bjp_group_year_sum["cited_by_count"].to_numpy(), #or normalized
    label="BJP",
    color="tab:orange",
    linewidth=2
)

plt.xlabel("Year", fontsize=14)
plt.ylabel("Number of Citations", fontsize=14)
plt.legend(fontsize=12)
plt.title("Number of Citations Over Time - Q1 vs BJP", fontsize=16)
plt.grid(alpha=0.3)
plt.xlim(1950, 2024)
plt.show()

max_q1_year, max_q1_value = (
    group_year_sum
    .select(["year", "cited_by_count"])
    .sort("cited_by_count", descending=True)
    .row(0)
)

print(max_q1_year)
print(max_q1_value)


max_bjp_year, max_bjp_value = (
    bjp_group_year_sum
    .select(["year", "cited_by_count"])
    .sort("cited_by_count", descending=True)
    .row(0)
)

print(max_bjp_year)
print(max_bjp_value)




In [ ]:
plt.figure(figsize=(10, 6))


plt.plot(
    group_year_sum["year"].to_numpy(),
    group_year_sum["referenced_works_count"].to_numpy(),#or normalized
    label="Q1",
    color="tab:blue",
    linewidth=2
)

plt.plot(
    bjp_group_year_sum["year"].to_numpy(),
    bjp_group_year_sum["referenced_works_count"].to_numpy(),#or normalized
    label="BJP",
    color="tab:orange",
    linewidth=2
)

plt.xlabel("Year of publication", fontsize=14)
plt.legend(fontsize=12)
plt.xlabel("Year")
plt.ylabel("Number of References")
plt.title("Number of Referenced Works Over Time - Q1 vs BJP", fontsize=16)
plt.grid(alpha=0.3)
plt.xlim(1950, 2024)
plt.show()

max_q1_year, max_q1_value = (
    group_year_sum
    .select(["year", "referenced_works_count"])
    .sort("referenced_works_count", descending=True)
    .row(0)
)

print(max_q1_year)
print(max_q1_value)


max_bjp_year, max_bjp_value = (
    bjp_group_year_sum
    .select(["year", "referenced_works_count"])
    .sort("referenced_works_count", descending=True)
    .row(0)
)

print(max_bjp_year)
print(max_bjp_value)

In [ ]:
#years_q1_ci = group_year_q1["by"].to_numpy()
#mean_q1_ci = group_year_q1["cited_by_count_mean"].to_numpy()
#median_q1_ci = group_year_q1["mncs_median"].to_numpy()
#low_q1 = group_year_q1["cited_by_count_p2_5"].to_numpy()
#high_q1 = group_year_q1["cited_by_count_p97_5"].to_numpy()

years_bjp_ci = group_year_bjp["by"].to_numpy()
mean_bjp_ci = group_year_bjp["mncs_mean"].to_numpy()
#median_bjp_ci = group_year_bjp["mncs_median"].to_numpy()
#low_bjp = group_year_bjp["cited_by_count_p2_5"].to_numpy()
#high_bjp = group_year_bjp["cited_by_count_p97_5"].to_numpy()

with aq.load_theme("scientific"):

    plt.figure(figsize=(7.2, 6))

    # Ligne horizontale MNCS = 1
    plt.axhline(y=1, color="black", linestyle="--", linewidth=1, label="MNCS = 1")

    # Courbe BJP
    plt.plot(years_bjp_ci, mean_bjp_ci, color="#f39c12", label="BJP mean")

    plt.xlabel("Year of Publication", fontsize=16)
    plt.ylabel("Number of Citations", fontsize=16)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    plt.legend(fontsize=14)
    plt.title("MNCS Per Work Over Time - BJP Journal", fontsize=18)
    plt.show()



In [ ]:
years = list(range(2012, 2025))
cited_cols = [f"cited_by_count_{year}_sum" for year in years]

df = group_journals.to_pandas()

top1 = []
top5 = []
top10 = []
mean_q1 = []
bjp = df[df["by"] == "British Journal of Pharmacology"][cited_cols].iloc[0]

for col in cited_cols:
    sorted_year = df.sort_values(col, ascending=False)[col]
    mean_q1.append(sorted_year.mean())
    top1.append(sorted_year.iloc[0])
    top5.append(sorted_year.iloc[4])
    top10.append(sorted_year.iloc[9])
    
plt.figure(figsize=(12, 6))
sns.set_style("whitegrid")

plt.plot(years, mean_q1, label='Q1 Average', linewidth=2)
plt.plot(years, top1, label='1st', linewidth=2)
plt.plot(years, top5, label='5th', linewidth=2)
plt.plot(years, top10, label='10th', linewidth=2)
plt.plot(years, bjp, label='BJP', linewidth=2, color='red')

plt.title('Citations by Year (2012–2024) for Q1 Journals and BJP')
plt.xlabel('Year')
plt.ylabel('Number of Citations')
plt.xticks(years, rotation=45)
plt.legend()
plt.tight_layout()
plt.show()



In [ ]:
means_yearly_citations = [works[f"cited_by_count_{i}"].mean() for i in range(2012, 2025)]

new_studies = works.filter(pl.col("year") == 2012)
means_2012_yearly_citations = [new_studies[f"cited_by_count_{i}"].mean() for i in range(2012, 2025)]

years = range(13)
years_2012 = range(2012, 2025)

list_means_yearly_citations = []
for year in years:
    new_studies = works.filter(pl.col("year") == 2012 + year)
    list_means_yearly_citations.append([new_studies[f"cited_by_count_{i}"].mean() for i in range(2012 + year, 2025)])

In [ ]:
means_yearly_citations_bjp = [works_bjp[f"cited_by_count_{i}"].mean() for i in range(2012, 2025)]

new_studies_bjp = works.filter(pl.col("year") == 2012)
means_2012_yearly_citations_bjp = [new_studies_bjp[f"cited_by_count_{i}"].mean() for i in range(2012, 2025)]

list_means_yearly_citations_bjp = []
for year in years:
    new_studies_bjp = works_bjp.filter(pl.col("year") == 2012 + year)
    list_means_yearly_citations_bjp.append([new_studies_bjp[f"cited_by_count_{i}"].mean() for i in range(2012 + year, 2025)])

In [ ]:
with aq.load_theme("scientific"):
    plt.figure(figsize=(7.2, 6))
    sns.lineplot(x = years_2012, y =  means_yearly_citations_bjp, label = "all studies")
    sns.lineplot(x = years_2012, y =  means_2012_yearly_citations_bjp, label = "studies published in 2012")
    plt.xlabel("Year", fontsize=18)
    plt.ylabel("Mean Citations", fontsize=18)
    plt.title("Yearly Citations: 2012 BJP Publications vs All BJP Publications")
    plt.xticks(fontsize=18)
    plt.yticks(fontsize=18)
    plt.legend(fontsize=18)
    plt.show()

In [ ]:
years = list(range(2012, 2025))
cols_cited = [f"cited_by_count_{y}" for y in years]

def yearly_citation_stats(df, cols):
    agg_exprs = []
    for col in cols:
        agg_exprs.append(pl.mean(col).alias(f"{col}_mean"))
        agg_exprs.append(pl.median(col).alias(f"{col}_median"))
        agg_exprs.append(pl.col(col).quantile(0.025).alias(f"{col}_p2_5"))
        agg_exprs.append(pl.col(col).quantile(0.975).alias(f"{col}_p97_5"))
    return df.select(agg_exprs)

stats_q1 = yearly_citation_stats(works, cols_cited)
stats_bjp = yearly_citation_stats(works_bjp, cols_cited)

mean_q1 = [stats_q1[f"{c}_mean"][0] for c in cols_cited]
median_q1 = [stats_q1[f"{c}_median"][0] for c in cols_cited]
low_q1 = [stats_q1[f"{c}_p2_5"][0] for c in cols_cited]
high_q1 = [stats_q1[f"{c}_p97_5"][0] for c in cols_cited]

mean_bjp = [stats_bjp[f"{c}_mean"][0] for c in cols_cited]
median_bjp = [stats_bjp[f"{c}_median"][0] for c in cols_cited]
low_bjp = [stats_bjp[f"{c}_p2_5"][0] for c in cols_cited]
high_bjp = [stats_bjp[f"{c}_p97_5"][0] for c in cols_cited]

with aq.load_theme("scientific"):
    plt.figure(figsize=(12,6))

    plt.plot(years, mean_q1, color="#3944f3", label="Q1 mean")
    #plt.plot(years, median_q1, color="#39daf3", label="Q1 median")
    #plt.fill_between(years, low_q1, high_q1, color="#3944f3", alpha=0.2)

    plt.plot(years, mean_bjp, color="#f39c12", label="BJP mean")
    #plt.plot(years, median_bjp, color="#f31212", label="BJP median")
    #plt.fill_between(years, low_bjp, high_bjp, color="#f39c12", alpha=0.2)

    plt.xlabel("Year", fontsize=14)
    plt.ylabel("Citations per Year", fontsize=14)
    plt.title("Annual Citations Dynamics - Q1 vs BJP", fontsize=16)
    plt.legend(fontsize=12)
    plt.grid(alpha=0.3)
    plt.show()


In [ ]:
number_studies_per_year = (
    works
    .with_columns(pl.lit(1).alias("count"))
    .select(["year", "count"])
    .group_by("year")
    .sum()
    .sort("year")
)

number_studies_per_year_series = pd.Series(
    number_studies_per_year["count"].to_list(),
    index=number_studies_per_year["year"].to_list()
)
number_studies_per_year_series = number_studies_per_year_series.reindex(years_2012, fill_value=0)

number_studies_per_year_series = (number_studies_per_year_series - number_studies_per_year_series.min()) / \
                                 (number_studies_per_year_series.max() - number_studies_per_year_series.min())

means_yearly_citations = []
for year in years_2012:
    new_studies = works.filter(pl.col("year") == year)
    mean_cites = [new_studies[f"cited_by_count_{i}"].mean() for i in range(year, 2025)]
    means_yearly_citations.append(sum(mean_cites)/len(mean_cites)) 

means_yearly_citations_series = pd.Series(means_yearly_citations, index=years_2012)
means_yearly_citations_series = (means_yearly_citations_series - means_yearly_citations_series.min()) / \
                                (means_yearly_citations_series.max() - means_yearly_citations_series.min())


In [ ]:
number_studies_per_year_bjp = (
    works_bjp
    .with_columns(pl.lit(1).alias("count"))
    .select(["year", "count"])
    .group_by("year")
    .sum()
    .sort("year")
)

number_studies_per_year_series_bjp = pd.Series(
    number_studies_per_year_bjp["count"].to_list(),
    index=number_studies_per_year_bjp["year"].to_list()
)
number_studies_per_year_series_bjp = number_studies_per_year_series_bjp.reindex(years_2012, fill_value=0)

number_studies_per_year_series_bjp = (number_studies_per_year_series_bjp - number_studies_per_year_series_bjp.min()) / \
                                 (number_studies_per_year_series_bjp.max() - number_studies_per_year_series_bjp.min())

means_yearly_citations_bjp = []
for year in years_2012:
    new_studies_bjp = works_bjp.filter(pl.col("year") == year)
    mean_cites_bjp = [new_studies_bjp[f"cited_by_count_{i}"].mean() for i in range(year, 2025)]
    means_yearly_citations_bjp.append(sum(mean_cites_bjp)/len(mean_cites_bjp)) 

means_yearly_citations_series_bjp = pd.Series(means_yearly_citations_bjp, index=years_2012)
means_yearly_citations_series_bjp = (means_yearly_citations_series_bjp - means_yearly_citations_series_bjp.min()) / \
                                (means_yearly_citations_series_bjp.max() - means_yearly_citations_series_bjp.min())


In [ ]:
with aq.load_theme("scientific"):
    plt.figure(figsize=(7.2, 6))
    for i in range(13):
        y = list_means_yearly_citations[i]
        sns.lineplot(x = years[:len(y)], y = y, label = f"{2012 + i}", color = sns.set_palette("viridis", n_colors = 13))
    plt.xlabel("Age", fontsize=18)
    plt.ylabel("Mean Citations", fontsize=18)
    plt.legend()
    plt.xticks(fontsize=18)
    plt.yticks(fontsize=18)
    plt.title("Evolution of Citations Over Study Age - Q1 Journals")
    plt.show()

In [ ]:
with aq.load_theme("scientific"):
    plt.figure(figsize=(7.2, 6))
    for i in range(13):
        y = list_means_yearly_citations_bjp[i]
        sns.lineplot(x = years[:len(y)], y = y, label = f"{2012 + i}", color = sns.set_palette("viridis", n_colors = 13))
    plt.xlabel("Age", fontsize=18)
    plt.ylabel("Mean Citations", fontsize=18)
    plt.legend()
    plt.xticks(fontsize=18)
    plt.yticks(fontsize=18)
    plt.title("Evolution of Citations Over Study Age - BJP Journal")
    plt.show()

In [ ]:
import ast

works = works.with_columns([
    pl.Series([ast.literal_eval(x)["value"] if x else None for x in works["citation_normalized_percentile"]]).alias("citation_percentile_value"),
    pl.Series([ast.literal_eval(x)["is_in_top_1_percent"] if x else None for x in works["citation_normalized_percentile"]]).alias("is_top_1_percent"),
    pl.Series([ast.literal_eval(x)["is_in_top_10_percent"] if x else None for x in works["citation_normalized_percentile"]]).alias("is_top_10_percent")
])

works_bjp = works_bjp.with_columns([
    pl.Series([ast.literal_eval(x)["value"] if x else None for x in works_bjp["citation_normalized_percentile"]]).alias("citation_percentile_value"),
    pl.Series([ast.literal_eval(x)["is_in_top_1_percent"] if x else None for x in works_bjp["citation_normalized_percentile"]]).alias("is_top_1_percent"),
    pl.Series([ast.literal_eval(x)["is_in_top_10_percent"] if x else None for x in works_bjp["citation_normalized_percentile"]]).alias("is_top_10_percent")
])



In [ ]:
works_bjp.filter(pl.col("is_top_10_percent") == True,
                 (pl.col("year") >= 2020))

In [ ]:
works_bjp.filter(pl.col("is_top_1_percent") == True,
                 (pl.col("year") >= 2020))

In [ ]:
agg_percentile = [
    pl.mean("citation_percentile_value").alias("mean"),
    pl.median("citation_percentile_value").alias("median"),
    pl.col("citation_percentile_value").quantile(0.025).alias("p2_5"),
    pl.col("citation_percentile_value").quantile(0.975).alias("p97_5"),
]

group_q1_perc = works.group_by("year", maintain_order=True).agg(agg_percentile).sort("year")
group_bjp_perc = works_bjp.group_by("year", maintain_order=True).agg(agg_percentile).sort("year")


years_q1 = group_q1_perc["year"].to_numpy()
mean_q1 = group_q1_perc["mean"].to_numpy()
median_q1 = group_q1_perc["median"].to_numpy()
low_q1 = group_q1_perc["p2_5"].to_numpy()
high_q1 = group_q1_perc["p97_5"].to_numpy()

years_bjp = group_bjp_perc["year"].to_numpy()
mean_bjp = group_bjp_perc["mean"].to_numpy()
median_bjp = group_bjp_perc["median"].to_numpy()
low_bjp = group_bjp_perc["p2_5"].to_numpy()
high_bjp = group_bjp_perc["p97_5"].to_numpy()

with aq.load_theme("scientific"):
    plt.figure(figsize=(8,6))

    plt.plot(years_q1, mean_q1, color="#3944f3", label="Q1 mean")
    #plt.plot(years_q1, median_q1, color="#39daf3", label="Q1 median")
    #plt.fill_between(years_q1, low_q1, high_q1, color="#3944f3", alpha=0.2)

    plt.plot(years_bjp, mean_bjp, color="#f39c12", label="BJP mean")
    #plt.plot(years_bjp, median_bjp, color="#f31212", label="BJP median")
    #plt.fill_between(years_bjp, low_bjp, high_bjp, color="#f39c12", alpha=0.2)

    plt.xlabel("Year of Publication", fontsize=14)
    plt.ylabel("Citation Normalized Percentile", fontsize=14)
    plt.title("Citation Percentile by Year - Q1 vs BJP", fontsize=16)
    plt.legend(fontsize=12)
    plt.grid(alpha=0.3)
    plt.xlim(1950, 2024)
    plt.ylim(0, 1)  
    plt.show()

In [ ]:


def top10_percent_per_year(df):
    grouped = (
        df.group_by("year", maintain_order=True)
          .agg([
              (pl.col("is_top_10_percent").sum() / pl.count() * 100).alias("pct_top10")
          ])
          .sort("year")
    )
    return grouped

q1_top10_year = top10_percent_per_year(works)
bjp_top10_year = top10_percent_per_year(works_bjp)

with aq.load_theme("scientific"):
    plt.figure(figsize=(10,6))
    plt.plot(q1_top10_year["year"].to_numpy(), q1_top10_year["pct_top10"].to_numpy(),
             color="#3944f3", label="Q1 % top 10%", linewidth=2)
    plt.plot(bjp_top10_year["year"].to_numpy(), bjp_top10_year["pct_top10"].to_numpy(),
             color="#f39c12", label="BJP % top 10%", linewidth=2)
    plt.xlabel("Year of Publication", fontsize=14)
    plt.ylabel("% of Articles in Top 10% Citations", fontsize=14)
    plt.title("Percentage of Articles in Top 10% by Normalized Citations Over Time", fontsize=16)
    plt.legend(fontsize=12)
    plt.grid(alpha=0.3)
    plt.xlim(1950, 2024)
    plt.show()

In [ ]:
def top1_percent_per_year(df):
    grouped = (
        df.group_by("year", maintain_order=True)
          .agg([
              (pl.col("is_top_1_percent").sum() / pl.len() * 100).alias("pct_top1")
          ])
          .sort("year")
    )
    return grouped

q1_top1_year = top1_percent_per_year(works)
bjp_top1_year = top1_percent_per_year(works_bjp)
print(q1_top1_year)

with aq.load_theme("scientific"):
    plt.figure(figsize=(10,6))
    plt.plot(q1_top1_year["year"].to_numpy(), q1_top1_year["pct_top1"].to_numpy(),
             color="#3944f3", label="Q1 % top 1%", linewidth=2)
    plt.plot(bjp_top1_year["year"].to_numpy(), bjp_top1_year["pct_top1"].to_numpy(),
             color="#f39c12", label="BJP % top 1%", linewidth=2)
    plt.xlabel("Year of Publication", fontsize=14)
    plt.ylabel("% of Articles in Top 1% Citations", fontsize=14)
    plt.title("Percentage of Articles in Top 1% by Normalized Citations Over Time", fontsize=16)
    plt.legend(fontsize=12)
    plt.grid(alpha=0.3)
    plt.xlim(1950, 2024)    
    plt.show()

In [ ]:

total_q1 = works.height
top1_q1 = works.filter(pl.col("is_top_1_percent") == True).height
top10_q1 = works.filter(pl.col("is_top_10_percent") == True).height

pct_top1_q1 = top1_q1 / total_q1 * 100
pct_top10_q1 = top10_q1 / total_q1 * 100

print(f"Q1: {pct_top1_q1:.2f}% of articles in top 1%")
print(f"Q1: {pct_top10_q1:.2f}% of articles in top 10%")

total_bjp = works_bjp.height
top1_bjp = works_bjp.filter(pl.col("is_top_1_percent") == True).height
top10_bjp = works_bjp.filter(pl.col("is_top_10_percent") == True).height

pct_top1_bjp = top1_bjp / total_bjp * 100
pct_top10_bjp = top10_bjp / total_bjp * 100

print(f"BJP: {pct_top1_bjp:.2f}% of articles in top 1%")
print(f"BJP: {pct_top10_bjp:.2f}% of articles in top 10%")


In [ ]:
def h_index_array(citations):
    if len(citations) == 0:
        return 0
    citations_sorted = np.sort(citations)[::-1]
    h = np.arange(1, len(citations_sorted)+1)
    return int(np.max(np.minimum(citations_sorted, h)))

def h_index_yearly(df):
    years = sorted(df["year"].unique().to_list())
    h_vals = []

    for y in years:
        citations = np.array(df.filter(pl.col("year") == y)["cited_by_count"].to_list())
        h_vals.append(h_index_array(citations))

    h_vals = np.array(h_vals)

    if len(h_vals) > 0:
        h_max = h_vals.max()
        year_max = years[h_vals.argmax()]   

    print("h-index max :", h_max)
    print("Année du h-index max :", year_max)

    return years, h_vals


years_q1, h_q1 = h_index_yearly(works)
years_bjp, h_bjp = h_index_yearly(works_bjp)

with aq.load_theme("scientific"):
    plt.figure(figsize=(10,6))
    plt.plot(years_q1, h_q1, color="#3944f3", label="Q1 H-index", linewidth=2)
    plt.plot(years_bjp, h_bjp, color="#f39c12", label="BJP H-index", linewidth=2)
    plt.xlabel("Year of Publication", fontsize=14)
    plt.ylabel("H-index", fontsize=14)
    plt.title("Annual H-index - Q1 vs BJP", fontsize=16)
    plt.legend(fontsize=12)
    plt.grid(alpha=0.3)
    plt.xlim(1950, 2024)
    plt.show()

In [ ]:
works = works.with_columns(
    (pl.col("cited_by_count") / pl.col("age")).alias("cited_per_year")
)

works_bjp = works_bjp.with_columns(
    (pl.col("cited_by_count") / pl.col("age")).alias("cited_per_year")
)

agg_exprs_age = [
    pl.mean("cited_per_year").alias("mean"),
    pl.median("cited_per_year").alias("median"),
    pl.col("cited_per_year").quantile(0.025).alias("p2_5"),
    pl.col("cited_per_year").quantile(0.975).alias("p97_5"),
]

group_q1_age = (
    works.group_by("year", maintain_order=True)
    .agg(agg_exprs_age)
    .sort("year")
)

group_bjp_age = (
    works_bjp.group_by("year", maintain_order=True)
    .agg(agg_exprs_age)
    .sort("year")
)

years_q1 = group_q1_age["year"].to_numpy()
mean_q1 = group_q1_age["mean"].to_numpy()
median_q1 = group_q1_age["median"].to_numpy()
low_q1 = group_q1_age["p2_5"].to_numpy()
high_q1 = group_q1_age["p97_5"].to_numpy()

years_bjp = group_bjp_age["year"].to_numpy()
mean_bjp = group_bjp_age["mean"].to_numpy()
median_bjp = group_bjp_age["median"].to_numpy()
low_bjp = group_bjp_age["p2_5"].to_numpy()
high_bjp = group_bjp_age["p97_5"].to_numpy()


with aq.load_theme("scientific"):
    plt.figure(figsize=(8,6))


    plt.plot(years_q1, mean_q1, color="#3944f3", label="Q1 mean")
    #plt.plot(years_q1, median_q1, color="#39daf3", label="Q1 median")
    #plt.fill_between(years_q1, low_q1, high_q1, color="#3944f3", alpha=0.2)

    plt.plot(years_bjp, mean_bjp, color="#f39c12", label="BJP mean")
    #plt.plot(years_bjp, median_bjp, color="#f31212", label="BJP median")
    #plt.fill_between(years_bjp, low_bjp, high_bjp, color="#f39c12", alpha=0.2)

    plt.xlabel("Year of publication", fontsize=14)
    plt.ylabel("Citations normalized by age", fontsize=14)
    plt.legend(fontsize=12)
    plt.grid(alpha=0.3)
    plt.title("Citations per Age Over Time - Q1 vs BJP", fontsize=18)
    plt.xlim(1950, 2024)
    plt.show()


In [ ]:
works_small = works.select([
    "referenced_works_count",
    "cited_by_count",
    "year",
    "mncs"
])

mask_refs = works_small["referenced_works_count"] > 0
mask_cits = works_small["cited_by_count"] > 0

works_with_references = works_small.filter(mask_refs)
works_with_citations = works_small.filter(mask_cits)

works_small_bjp = works_bjp.select([
    "referenced_works_count",
    "cited_by_count",
    "year",
    "mncs"
])

mask_refs_bjp = works_small_bjp["referenced_works_count"] > 0
mask_cits_bjp = works_small_bjp["cited_by_count"] > 0

works_with_references_bjp = works_small_bjp.filter(mask_refs_bjp)
works_with_citations_bjp = works_small_bjp.filter(mask_cits_bjp)


In [ ]:


cols_to_stats = [
    "cited_by_count",
    "referenced_works_count",
]

agg_exprs = []

for col in cols_to_stats:
    agg_exprs.append(pl.mean(col).alias(f"{col}_mean"))
    agg_exprs.append(pl.median(col).alias(f"{col}_median"))
    agg_exprs.append(pl.col(col).quantile(0.025).alias(f"{col}_p2_5"))
    agg_exprs.append(pl.col(col).quantile(0.975).alias(f"{col}_p97_5"))
    agg_exprs.append(pl.sum(col).alias(f"{col}_sum"))
    
group_year_cbjp = (
    works_with_citations_bjp.group_by(
        by=pl.col("year"),
        maintain_order=True
    )
    .agg(agg_exprs)
)

group_year_rbjp = (
    works_with_references_bjp.group_by(
        by=pl.col("year"),
        maintain_order=True
    )
    .agg(agg_exprs)
)

In [ ]:


group_year_c = (
    works_with_citations.group_by(
        by=pl.col("year"),
        maintain_order=True
    )
    .agg(agg_exprs)
)

group_year_r = (
    works_with_references.group_by(
        by=pl.col("year"),
        maintain_order=True
    )
    .agg(agg_exprs)
)

group_year_c = group_year_c.sort("by", descending=False)
group_year_r = group_year_r.sort("by", descending=False)
group_year_cbjp = group_year_cbjp.sort("by", descending=False)
group_year_rbjp = group_year_rbjp.sort("by", descending=False)

In [ ]:
years_q1_ci = group_year_q1["by"].to_numpy()
mean_q1_ci = group_year_q1["cited_by_count_mean"].to_numpy()
median_q1_ci = group_year_q1["cited_by_count_median"].to_numpy()
low_q1 = group_year_q1["cited_by_count_p2_5"].to_numpy()
high_q1 = group_year_q1["cited_by_count_p97_5"].to_numpy()

years_bjp_ci = group_year_bjp["by"].to_numpy()
mean_bjp_ci = group_year_bjp["cited_by_count_mean"].to_numpy()
median_bjp_ci = group_year_bjp["cited_by_count_median"].to_numpy()
low_bjp = group_year_bjp["cited_by_count_p2_5"].to_numpy()
high_bjp = group_year_bjp["cited_by_count_p97_5"].to_numpy()

with aq.load_theme("scientific"):

    plt.figure(figsize=(7.2, 6))

    plt.plot(years_q1_ci, mean_q1_ci, color="#3944f3", label="Q1 works mean")
    #plt.plot(years_q1_ci, median_q1_ci, color="#39daf3", label="Q1 works median")
    #plt.fill_between(years_q1_ci, low_q1, high_q1, color="#3944f3", alpha=0.2)

    plt.plot(years_bjp_ci, mean_bjp_ci, color="#f39c12", label="BJP works mean")
    #plt.plot(years_bjp_ci, median_bjp_ci, color="#f31212", label="BJP works median")
    #plt.fill_between(years_bjp_ci, low_bjp, high_bjp, color="#f39c12", alpha=0.2)

    plt.xlabel("Year of Publication", fontsize=16)
    plt.ylabel("Number of Citations", fontsize=16)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    plt.legend(fontsize=14)
    plt.title("Citations Per Work Over Time - Q1 vs BJP", fontsize=18)
    plt.show()


In [ ]:
years_q1_ci = group_year_q1["by"].to_numpy()
mean_q1_ci = group_year_q1["referenced_works_count_mean"].to_numpy()
median_q1_ci = group_year_q1["referenced_works_count_median"].to_numpy()
low_q1 = group_year_q1["referenced_works_count_p2_5"].to_numpy()
high_q1 = group_year_q1["referenced_works_count_p97_5"].to_numpy()

years_bjp_ci = group_year_bjp["by"].to_numpy()
mean_bjp_ci = group_year_bjp["referenced_works_count_mean"].to_numpy()
median_bjp_ci = group_year_bjp["referenced_works_count_median"].to_numpy()
low_bjp = group_year_bjp["referenced_works_count_p2_5"].to_numpy()
high_bjp = group_year_bjp["referenced_works_count_p97_5"].to_numpy()

with aq.load_theme("scientific"):
    plt.figure(figsize=(7.2, 6))

    plt.plot(years_q1_ci, mean_q1_ci, color="#3944f3", label="Q1 works mean")
    #plt.plot(years_q1_ci, median_q1_ci, color="#39daf3", label="Q1 works median")
    #plt.fill_between(years_q1_ci, low_q1, high_q1, color="#3944f3", alpha=0.2)

    plt.plot(years_bjp_ci, mean_bjp_ci, color="#f39c12", label="BJP works mean")
    #plt.plot(years_bjp_ci, median_bjp_ci, color="#f31212", label="BJP works median")
    #plt.fill_between(years_bjp_ci, low_bjp, high_bjp, color="#f39c12", alpha=0.2)

    plt.xlabel("Year of Publication", fontsize=16)
    plt.ylabel("Number of Referenced Works", fontsize=16)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    plt.legend(fontsize=14)
    plt.title("Referenced Works Per Work Over Time - Q1 vs BJP", fontsize=18)
    plt.show()


In [ ]:
years_q1_ci = group_year_c["by"].to_numpy()
mean_q1_ci = group_year_c["cited_by_count_mean"].to_numpy()
median_q1_ci = group_year_c["cited_by_count_median"].to_numpy()
low_q1 = group_year_c["cited_by_count_p2_5"].to_numpy()
high_q1 = group_year_c["cited_by_count_p97_5"].to_numpy()

years_bjp_ci = group_year_cbjp["by"].to_numpy()
mean_bjp_ci = group_year_cbjp["cited_by_count_mean"].to_numpy()
median_bjp_ci = group_year_cbjp["cited_by_count_median"].to_numpy()
low_bjp = group_year_cbjp["cited_by_count_p2_5"].to_numpy()
high_bjp = group_year_cbjp["cited_by_count_p97_5"].to_numpy()

with aq.load_theme("scientific"):
    plt.figure(figsize=(7.2, 6))

    plt.plot(years_q1_ci, mean_q1_ci, color="#3944f3", label="Q1 works mean")
    #plt.plot(years_q1_ci, median_q1_ci, color="#39daf3", label="Q1 works median")
    #plt.fill_between(years_q1_ci, low_q1, high_q1, color="#3944f3", alpha=0.2)

    plt.plot(years_bjp_ci, mean_bjp_ci, color="#f39c12", label="BJP works mean")
    #plt.plot(years_bjp_ci, median_bjp_ci, color="#f31212", label="BJP works median")
    #plt.fill_between(years_bjp_ci, low_bjp, high_bjp, color="#f39c12", alpha=0.2)

    plt.xlabel("Year of Publication", fontsize=16)
    plt.ylabel("Number of Citations", fontsize=16)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    plt.xlim(1950, 2024)
    plt.legend(fontsize=14)
    plt.title("Citations Per Work (>0 Citations) Over Time - Q1 vs BJP", fontsize=18)
    plt.show()


In [ ]:
years_q1_ci = group_year_r["by"].to_numpy()
mean_q1_ci = group_year_r["referenced_works_count_mean"].to_numpy()
median_q1_ci = group_year_r["referenced_works_count_median"].to_numpy()
low_q1 = group_year_r["referenced_works_count_p2_5"].to_numpy()
high_q1 = group_year_r["referenced_works_count_p97_5"].to_numpy()

years_bjp_ci = group_year_rbjp["by"].to_numpy()
mean_bjp_ci = group_year_rbjp["referenced_works_count_mean"].to_numpy()
median_bjp_ci = group_year_rbjp["referenced_works_count_median"].to_numpy()
low_bjp = group_year_rbjp["referenced_works_count_p2_5"].to_numpy()
high_bjp = group_year_rbjp["referenced_works_count_p97_5"].to_numpy()

with aq.load_theme("scientific"):
    plt.figure(figsize=(7.2, 6))

    plt.plot(years_q1_ci, mean_q1_ci, color="#3944f3", label="Q1 works mean")
    #plt.plot(years_q1_ci, median_q1_ci, color="#39daf3", label="Q1 works median")
    #plt.fill_between(years_q1_ci, low_q1, high_q1, color="#3944f3", alpha=0.2)

    plt.plot(years_bjp_ci, mean_bjp_ci, color="#f39c12", label="BJP works mean")
    #plt.plot(years_bjp_ci, median_bjp_ci, color="#f31212", label="BJP works median")
    #plt.fill_between(years_bjp_ci, low_bjp, high_bjp, color="#f39c12", alpha=0.2)

    plt.xlabel("Year of Publication", fontsize=16)
    plt.ylabel("Number of Referenced Works", fontsize=16)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    plt.xlim(1950, 2024)
    plt.legend(fontsize=14)
    plt.title("Referenced Works Per Work (>0 References) Over Time - Q1 vs BJP", fontsize=18)
    plt.show()


In [ ]:
years_q1_ci = group_year_q1["by"].to_numpy()
mean_q1_ci = group_year_q1["authors_count_mean"].to_numpy()
median_q1_ci = group_year_q1["authors_count_median"].to_numpy()
low_q1 = group_year_q1["authors_count_p2_5"].to_numpy()
high_q1 = group_year_q1["authors_count_p97_5"].to_numpy()

years_bjp_ci = group_year_bjp["by"].to_numpy()
mean_bjp_ci = group_year_bjp["authors_count_mean"].to_numpy()
median_bjp_ci = group_year_bjp["authors_count_median"].to_numpy()
low_bjp = group_year_bjp["authors_count_p2_5"].to_numpy()
high_bjp = group_year_bjp["authors_count_p97_5"].to_numpy()

with aq.load_theme("scientific"):
    plt.figure(figsize=(7.2, 6))

    plt.plot(years_q1_ci, mean_q1_ci, color="#3944f3", label="Q1 works mean")
    #plt.plot(years_q1_ci, median_q1_ci, color="#39daf3", label="Q1 works median")
    #plt.fill_between(years_q1_ci, low_q1, high_q1, color="#3944f3", alpha=0.2)

    plt.plot(years_bjp_ci, mean_bjp_ci, color="#f39c12", label="BJP works mean")
    #plt.plot(years_bjp_ci, median_bjp_ci, color="#f31212", label="BJP works median")
    #plt.fill_between(years_bjp_ci, low_bjp, high_bjp, color="#f39c12", alpha=0.2)

    plt.xlabel("Year of Publication", fontsize=16)
    plt.ylabel("Number of Authors", fontsize=16)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    plt.legend(fontsize=14)
    plt.title("Authors Per Work Over Time - Q1 vs BJP", fontsize=18)
    plt.xlim(1950, 2024)
    plt.show()

In [ ]:
years_q1_ci = group_year_q1["by"].to_numpy()
mean_q1_ci = group_year_q1["countries_distinct_count_mean"].to_numpy()
median_q1_ci = group_year_q1["countries_distinct_count_median"].to_numpy()
low_q1 = group_year_q1["countries_distinct_count_p2_5"].to_numpy()
high_q1 = group_year_q1["countries_distinct_count_p97_5"].to_numpy()

years_bjp_ci = group_year_bjp["by"].to_numpy()
mean_bjp_ci = group_year_bjp["countries_distinct_count_mean"].to_numpy()
median_bjp_ci = group_year_bjp["countries_distinct_count_median"].to_numpy()
low_bjp = group_year_bjp["countries_distinct_count_p2_5"].to_numpy()
high_bjp = group_year_bjp["countries_distinct_count_p97_5"].to_numpy()

with aq.load_theme("scientific"):
    plt.figure(figsize=(7.2, 6))

    plt.plot(years_q1_ci, mean_q1_ci, color="#3944f3", label="Q1 works mean")
    #plt.plot(years_q1_ci, median_q1_ci, color="#39daf3", label="Q1 works median")
    #plt.fill_between(years_q1_ci, low_q1, high_q1, color="#3944f3", alpha=0.2)

    plt.plot(years_bjp_ci, mean_bjp_ci, color="#f39c12", label="BJP works mean")
    #plt.plot(years_bjp_ci, median_bjp_ci, color="#f31212", label="BJP works median")
    #plt.fill_between(years_bjp_ci, low_bjp, high_bjp, color="#f39c12", alpha=0.2)

    plt.xlabel("Year of Publication", fontsize=16)
    plt.ylabel("Number of Countries", fontsize=16)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    plt.legend(fontsize=14)
    plt.title("Countries Per Work Over Time - Q1 vs BJP", fontsize=18)
    plt.xlim(1950, 2024)
    plt.show()


In [ ]:
years_q1_ci = group_year_q1["by"].to_numpy()
mean_q1_ci = group_year_q1["institutions_distinct_count_mean"].to_numpy()
median_q1_ci = group_year_q1["institutions_distinct_count_mean"].to_numpy()
low_q1 = group_year_q1["institutions_distinct_count_p2_5"].to_numpy()
high_q1 = group_year_q1["institutions_distinct_count_p97_5"].to_numpy()

years_bjp_ci = group_year_bjp["by"].to_numpy()
mean_bjp_ci = group_year_bjp["institutions_distinct_count_mean"].to_numpy()
median_bjp_ci = group_year_bjp["institutions_distinct_count_median"].to_numpy()
low_bjp = group_year_bjp["institutions_distinct_count_p2_5"].to_numpy()
high_bjp = group_year_bjp["institutions_distinct_count_p97_5"].to_numpy()

with aq.load_theme("scientific"):
    plt.figure(figsize=(7.2, 6))

    plt.plot(years_q1_ci, mean_q1_ci, color="#3944f3", label="Q1 works mean")
    #plt.plot(years_q1_ci, median_q1_ci, color="#39daf3", label="Q1 works median")
    #plt.fill_between(years_q1_ci, low_q1, high_q1, color="#3944f3", alpha=0.2)

    plt.plot(years_bjp_ci, mean_bjp_ci, color="#f39c12", label="BJP works mean")
    #plt.plot(years_bjp_ci, median_bjp_ci, color="#f31212", label="BJP works median")
    #plt.fill_between(years_bjp_ci, low_bjp, high_bjp, color="#f39c12", alpha=0.2)

    plt.xlabel("Year of Publication", fontsize=16)
    plt.ylabel("Number of Insititutions", fontsize=16)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    plt.legend(fontsize=14)
    plt.title("Institutions Per Work Over Time - Q1 vs BJP", fontsize=18)
    plt.xlim(1950, 2024)
    plt.show()


In [ ]:
group_Nreferences = (
    works
    .group_by("referenced_works_count")
    .agg(*agg_exprs)
    .sort("referenced_works_count")
)

group_Nreferences_bjp = (
    works_bjp
    .group_by("referenced_works_count")
    .agg(*agg_exprs)
    .sort("referenced_works_count")
)

In [ ]:
r_pearson, p_pearson = st.pearsonr(works["referenced_works_count"], works["mncs"])
print(f"Pearson: r = {r_pearson:.3f}, p-value = {p_pearson:.3f}")

r_spearman, p_spearman = st.spearmanr(works["referenced_works_count"], works["mncs"])
print(f"Spearman: rho = {r_spearman:.3f}, p-value = {p_spearman:.3f}")

In [ ]:
references_q1_ci = group_Nreferences["referenced_works_count"].to_numpy()
mean_q1 = group_Nreferences["mncs_mean"].to_numpy()
median_q1 = group_Nreferences["mncs_median"].to_numpy()
low_q1 = group_Nreferences["mncs_p2_5"].to_numpy()
high_q1 = group_Nreferences["mncs_p97_5"].to_numpy()

references_bjp_ci = group_Nreferences_bjp["referenced_works_count"].to_numpy()
mean_bjp = group_Nreferences_bjp["mncs_mean"].to_numpy()
median_bjp = group_Nreferences_bjp["mncs_median"].to_numpy()
low_bjp = group_Nreferences_bjp["mncs_p2_5"].to_numpy()
high_bjp = group_Nreferences_bjp["mncs_p97_5"].to_numpy()


with aq.load_theme("scientific"):
    plt.figure(figsize=(7.2, 6))

    plt.plot(references_q1_ci, mean_q1, color="#3944f3", label="Q1 works mean")
    #plt.plot(references_q1_ci, median_q1, color="#39daf3",label="Q1 works median")
    #plt.fill_between(referencess_q1_ci, low_q1, high_q1, color="#3944f3", alpha=0.2)

    plt.plot(references_bjp_ci, mean_bjp, color="#f39c12", label="BJP works mean")
    #plt.plot(institutions_bjp_ci, median_bjp, color="#f31212",label="BJP works median")
    #plt.fill_between(references_bjp_ci, low_bjp, high_bjp, color="#f39c12", alpha=0.2)
    
    plt.xticks(rotation=45, fontsize=16)
    plt.yticks(fontsize=16)
    plt.xlabel("Number of references", fontsize=18)
    plt.ylabel("MNCS", fontsize=18)
    plt.xlim(0.9,85.1)
    plt.ylim(0, 3)
    plt.legend(fontsize=14)
    plt.title("MNCS by Number of References - Q1 vs BJP", fontsize=16)
    plt.show()